In [1]:
import torch

In [2]:
from pathlib import Path

## Import GPT2 model

In [3]:
Path.cwd()

PosixPath('/home/sameervk/Documents/Training/MachineLearning/LLM_app_huggingface/RnD/Training_and_Evaluation')

In [4]:
import sys

In [5]:
file = Path('__file__').resolve()

In [6]:
file

PosixPath('/home/sameervk/Documents/Training/MachineLearning/LLM_app_huggingface/RnD/Training_and_Evaluation/__file__')

In [7]:
parent, root = file.parent, file.parents[2]

In [8]:
parent

PosixPath('/home/sameervk/Documents/Training/MachineLearning/LLM_app_huggingface/RnD/Training_and_Evaluation')

In [9]:
root

PosixPath('/home/sameervk/Documents/Training/MachineLearning/LLM_app_huggingface')

In [10]:
sys.path.append(str(root))
try:
    sys.path.remove(str(parent))
except:
    pass

In [11]:
sys.path

['/home/sameervk/.local/share/uv/python/cpython-3.12.8-linux-x86_64-gnu/lib/python312.zip',
 '/home/sameervk/.local/share/uv/python/cpython-3.12.8-linux-x86_64-gnu/lib/python3.12',
 '/home/sameervk/.local/share/uv/python/cpython-3.12.8-linux-x86_64-gnu/lib/python3.12/lib-dynload',
 '',
 '/home/sameervk/Documents/Training/MachineLearning/LLM_app_huggingface/api/.venv/lib/python3.12/site-packages',
 '/home/sameervk/Documents/Training/MachineLearning/LLM_app_huggingface']

In [13]:
from RnD.LLM_arch.GPT2.llm_gpt2 import GPT2

## Import GPT2 config

In [14]:
import yaml

In [15]:
with open("../LLM_arch/GPT2/GPT2_arch_config.yaml", 'r') as file:
    config = yaml.safe_load(file.read())
    print(config)

{'vocab_size': 50257, 'context_length': 1024, 'embed_dim': 768, 'num_attention_heads': 12, 'num_layers': 12, 'drop_rate': 0.1, 'qkv_bias': False}


## Examination

In [16]:
test_model = GPT2(**config)

### Tokenizer

In [17]:
from tokenizers import Tokenizer

In [18]:
tokenizer = Tokenizer.from_pretrained("gpt2")

In [ ]:
del to

### Test inputs

In [19]:
test_data = ["The cost of training large language models is very high.",
            "This planet will fight back just like human bodies fight back against viruses."            
            ]

In [30]:
# encode
test_input_tokens = tokenizer.encode_batch(test_data, add_special_tokens=True)
test_input_tokens

[Encoding(num_tokens=11, attributes=[ids, type_ids, tokens, offsets, attention_mask, special_tokens_mask, overflowing]),
 Encoding(num_tokens=14, attributes=[ids, type_ids, tokens, offsets, attention_mask, special_tokens_mask, overflowing])]

In [24]:
vocab = tokenizer.get_vocab()

In [25]:
type(vocab)

dict

In [26]:
len(vocab)

50257

In [27]:
'<|endoftext|>' in vocab

True

In [28]:
vocab['<|endoftext|>']

50256

In [29]:
'<|unk|>' in vocab

False

In [32]:
tokenids = [elem.ids for elem in test_input_tokens]

In [33]:
tokenids

[[464, 1575, 286, 3047, 1588, 3303, 4981, 318, 845, 1029, 13],
 [1212,
  5440,
  481,
  1907,
  736,
  655,
  588,
  1692,
  5920,
  1907,
  736,
  1028,
  20547,
  13]]

In [34]:
test_x = [row[:4] for row in tokenids]
test_x

[[464, 1575, 286, 3047], [1212, 5440, 481, 1907]]

In [35]:
# y: shift 1 elem forward
test_y = [row[1:5] for row in tokenids]
test_y

[[1575, 286, 3047, 1588], [5440, 481, 1907, 736]]

### Model output probabilities

In [36]:
logits = test_model(torch.tensor(test_x))

In [37]:
logits.size()

torch.Size([2, 4, 50257])

In [101]:
probs = torch.softmax(logits, dim=-1)
probs.shape

torch.Size([2, 4, 50257])

#### Predicted Next Token Probabilities

In [71]:
predicted_probs = torch.max(probs, dim=-1)

In [72]:
predicted_probs

torch.return_types.max(
values=tensor([[0.0003, 0.0002, 0.0002, 0.0003],
        [0.0002, 0.0003, 0.0002, 0.0002]], grad_fn=<MaxBackward0>),
indices=tensor([[49633, 11466, 39003, 13462],
        [32524, 17028, 25234, 41271]]))

In [73]:
predicted_probs.values

tensor([[0.0003, 0.0002, 0.0002, 0.0003],
        [0.0002, 0.0003, 0.0002, 0.0002]], grad_fn=<MaxBackward0>)

In [74]:
predicted_probs.values.detach()

tensor([[0.0003, 0.0002, 0.0002, 0.0003],
        [0.0002, 0.0003, 0.0002, 0.0002]])

In [39]:
# output indices
output_idx = torch.argmax(probs, dim=-1, keepdim=True)
output_idx

tensor([[[49633],
         [11466],
         [39003],
         [13462]],

        [[32524],
         [17028],
         [25234],
         [41271]]])

#### Target probabilities

In [40]:
test_y

[[1575, 286, 3047, 1588], [5440, 481, 1907, 736]]

In [41]:
test_y = torch.tensor(test_y)
test_y.shape

torch.Size([2, 4])

In [61]:
probs[:,:,test_y]

tensor([[[[9.1741e-06, 2.7827e-05, 3.3129e-05, 6.5669e-06],
          [1.7419e-05, 1.7504e-05, 4.0294e-05, 2.0428e-05]],

         [[2.3878e-05, 4.4040e-05, 2.2334e-05, 1.4561e-05],
          [1.4114e-05, 5.3751e-06, 2.3284e-05, 1.3729e-05]],

         [[1.8311e-05, 1.2645e-05, 1.4060e-05, 1.3956e-05],
          [1.3896e-05, 5.8000e-06, 2.4935e-05, 1.8271e-05]],

         [[1.3174e-05, 1.8269e-05, 3.2913e-05, 1.9116e-05],
          [2.2694e-05, 1.5157e-05, 1.0626e-05, 1.3994e-05]]],


        [[[1.0472e-05, 2.8052e-05, 1.5445e-05, 9.4022e-06],
          [9.9974e-06, 1.2662e-05, 2.0372e-05, 2.9668e-05]],

         [[1.6634e-05, 2.7273e-05, 1.7232e-05, 1.3653e-05],
          [9.8789e-06, 6.6897e-06, 4.0113e-05, 1.4667e-05]],

         [[2.0394e-05, 4.5099e-05, 7.9016e-06, 8.8200e-06],
          [1.3738e-05, 1.1901e-05, 1.5592e-05, 2.0482e-05]],

         [[2.1178e-05, 2.1895e-05, 1.5412e-05, 1.3532e-05],
          [1.2191e-05, 1.0758e-05, 1.8833e-05, 1.1507e-05]]]],
       grad_fn=<Index

In [45]:
test_y.unsqueeze(-1).shape

torch.Size([2, 4, 1])

In [52]:
probs[0,[0,1,2,3], test_y[0]]

tensor([9.1741e-06, 4.4040e-05, 1.4060e-05, 1.9116e-05],
       grad_fn=<IndexBackward0>)

In [64]:
probs[1,[0,1,2,3], test_y[1]]

tensor([9.9974e-06, 6.6897e-06, 1.5592e-05, 1.1507e-05],
       grad_fn=<IndexBackward0>)

In [56]:
test_y[0].unsqueeze(-1)

tensor([[1575],
        [ 286],
        [3047],
        [1588]])

In [58]:
probs[0,:,test_y[0]]

tensor([[9.1741e-06, 2.7827e-05, 3.3129e-05, 6.5669e-06],
        [2.3878e-05, 4.4040e-05, 2.2334e-05, 1.4561e-05],
        [1.8311e-05, 1.2645e-05, 1.4060e-05, 1.3956e-05],
        [1.3174e-05, 1.8269e-05, 3.2913e-05, 1.9116e-05]],
       grad_fn=<IndexBackward0>)

In [65]:
probs[1,:,test_y[1]]

tensor([[9.9974e-06, 1.2662e-05, 2.0372e-05, 2.9668e-05],
        [9.8789e-06, 6.6897e-06, 4.0113e-05, 1.4667e-05],
        [1.3738e-05, 1.1901e-05, 1.5592e-05, 2.0482e-05],
        [1.2191e-05, 1.0758e-05, 1.8833e-05, 1.1507e-05]],
       grad_fn=<IndexBackward0>)

In [59]:
torch.diag(probs[0,:,test_y[0]], 0)

tensor([9.1741e-06, 4.4040e-05, 1.4060e-05, 1.9116e-05],
       grad_fn=<DiagonalBackward0_copy>)

In [63]:
[torch.diag(probs[i,:,test_y[i]], 0) for i in range(probs.shape[0])]

[tensor([9.1741e-06, 4.4040e-05, 1.4060e-05, 1.9116e-05],
        grad_fn=<DiagonalBackward0_copy>),
 tensor([9.9974e-06, 6.6897e-06, 1.5592e-05, 1.1507e-05],
        grad_fn=<DiagonalBackward0_copy>)]

In [66]:
x = torch.diag(probs[0,:,test_y[0]], 0)

In [67]:
x=x.detach()

In [68]:
x

tensor([9.1741e-06, 4.4040e-05, 1.4060e-05, 1.9116e-05])

In [69]:
[torch.diag(probs[i,:,test_y[i]], diagonal=0).detach() for i in range(probs.shape[0])]

[tensor([9.1741e-06, 4.4040e-05, 1.4060e-05, 1.9116e-05]),
 tensor([9.9974e-06, 6.6897e-06, 1.5592e-05, 1.1507e-05])]

In [75]:
target_probs = [torch.diag(probs[i,:,test_y[i]], diagonal=0).detach() for i in range(probs.shape[0])]
target_probs

[tensor([9.1741e-06, 4.4040e-05, 1.4060e-05, 1.9116e-05]),
 tensor([9.9974e-06, 6.6897e-06, 1.5592e-05, 1.1507e-05])]

In [77]:
predicted_probs.values.detach()

tensor([[0.0003, 0.0002, 0.0002, 0.0003],
        [0.0002, 0.0003, 0.0002, 0.0002]])

## Loss function

### NOTE

* Objective is to "to reduce the average log target probability to zero". Why?
* Answer: Because, loss calculation is the difference between true and predicted. So, this difference must be reduced to zero during training.

### Custom loss calculation

In [78]:
# concatenate
log_target_probs = torch.cat(target_probs)

In [79]:
log_target_probs

tensor([9.1741e-06, 4.4040e-05, 1.4060e-05, 1.9116e-05, 9.9974e-06, 6.6897e-06,
        1.5592e-05, 1.1507e-05])

In [80]:
# apply log in place
log_target_probs.log_()

tensor([-11.5991, -10.0304, -11.1722, -10.8650, -11.5132, -11.9149, -11.0687,
        -11.3726])

In [81]:
# mean
avg_log_target_probs = torch.mean(log_target_probs)
avg_log_target_probs

tensor(-11.1920)

In [82]:
# negative of mean
neg_avg_log_target_probs = -1*avg_log_target_probs
neg_avg_log_target_probs

tensor(11.1920)

### Cross Entropy

In [83]:
logits.shape

torch.Size([2, 4, 50257])

In [84]:
test_y.shape

torch.Size([2, 4])

In [87]:
test_y

tensor([[1575,  286, 3047, 1588],
        [5440,  481, 1907,  736]])

In [85]:
logits_flat = logits.flatten(0,1)
logits_flat.shape

torch.Size([8, 50257])

In [88]:
test_y_flat = test_y.flatten(0)
test_y_flat

tensor([1575,  286, 3047, 1588, 5440,  481, 1907,  736])

#### Using target indices

In [89]:
cross_entropy_loss = torch.nn.functional.cross_entropy(input=logits_flat, target=test_y_flat)
cross_entropy_loss

tensor(11.1920, grad_fn=<NllLossBackward0>)

* Matches the custom calculation

In [109]:
torch.max(logits_flat, dim=-1).values

tensor([2.7601, 2.4410, 2.2409, 2.8765, 2.3931, 2.7756, 2.4282, 2.4139],
       grad_fn=<MaxBackward0>)

#### Using probabilities

In [90]:
target_probs

[tensor([9.1741e-06, 4.4040e-05, 1.4060e-05, 1.9116e-05]),
 tensor([9.9974e-06, 6.6897e-06, 1.5592e-05, 1.1507e-05])]

In [95]:
target_probs_flat = torch.tensor([elem.tolist() for elem in target_probs])
target_probs_flat

tensor([[9.1741e-06, 4.4040e-05, 1.4060e-05, 1.9116e-05],
        [9.9974e-06, 6.6897e-06, 1.5592e-05, 1.1507e-05]])

In [96]:
target_probs_flat = target_probs_flat.flatten()
target_probs_flat

tensor([9.1741e-06, 4.4040e-05, 1.4060e-05, 1.9116e-05, 9.9974e-06, 6.6897e-06,
        1.5592e-05, 1.1507e-05])

In [110]:
cross_entropy_loss_p = torch.nn.functional.cross_entropy(torch.max(logits_flat, dim=-1).values, 
                                                         target_probs_flat,
                                                        )
cross_entropy_loss_p

tensor(0.0003, grad_fn=<DivBackward1>)

#### NOTE:
* Cannot use probabilities because we do not know the true probabilities.
* Here the true targets are class indices not probabilities.
* In the current check above using indices, pytorch uses the indices to retrieve the corresponding values from the predicted logits and follows the same steps as in the custom calculation.